<a href="https://colab.research.google.com/github/Quddos/machine-learning/blob/main/AP24135010028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Decision Tree Classifier for classification of Iris dataset***

In [8]:
# 1. Import required libraries
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
# a. Load the Iris dataset
iris = load_iris()
X = iris.data # Features
y = iris.target # Labels
# b. Split the dataset into train (70%) and test (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# c. Train a Decision Tree Classifier using train set
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
# d. Test the model using test set
y_pred = dt_model.predict(X_test)
# e. Find accuracy and confusion matrix
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix )
print("Accuracies of Decision Tree Classifier:", accuracy)

Confusion Matrix:
 [[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]
Accuracies of Decision Tree Classifier: 1.0
